In [35]:
import pandas as pd 
import numpy as np
import os

folder_path= './PassengerVehicle_Stats'
dfs=[]
files= [f for f in os.listdir(folder_path) if f.endswith('.csv')]

for file in files:
    file_path = os.path.join(folder_path,file)
    df1 = pd.read_csv(file_path)
    dfs.append(df1)

combine_df = pd.concat(dfs, ignore_index=True)
combine_df.to_csv('./vehicles_df.csv', index= False)

In [4]:
df = pd.read_csv('vehicles_df.csv')
df.isna().sum()

Unnamed: 0                                0
Public Vehicle Number                     0
Status                                    0
Vehicle Make                           7668
Vehicle Model                          7852
Vehicle Model Year                     7768
Vehicle Color                          7944
Vehicle Fuel Source                       0
Wheelchair Accessible                     0
Company Name                              0
Address                                7144
City                                   7144
State                                  7144
ZIP Code                               7144
Taxi Affiliation                      37016
Taxi Medallion License Management     37124
Record ID                                 0
dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66408 entries, 0 to 66407
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          66408 non-null  int64  
 1   Public Vehicle Number               66408 non-null  int64  
 2   Status                              66408 non-null  object 
 3   Vehicle Make                        58740 non-null  object 
 4   Vehicle Model                       58556 non-null  object 
 5   Vehicle Model Year                  58640 non-null  float64
 6   Vehicle Color                       58464 non-null  object 
 7   Vehicle Fuel Source                 66408 non-null  object 
 8   Wheelchair Accessible               66408 non-null  object 
 9   Company Name                        66408 non-null  object 
 10  Address                             59264 non-null  object 
 11  City                                59264

In [19]:
(df['Wheelchair Accessible']=='Y').sum()

2496

In [23]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [25]:
(df['Vehicle Model Year'].apply(lambda x: isinstance(x,float))).sum()

66408

In [118]:
df['Vehicle Model Year'].unique()

array([2014, 2010, 2008, 2015, 2012, 2016, 2011, 2017, 2009, 2013, 2024,
       2005, 2001, 2002, 2000, 1993, 2007, 1998, 2006, 1997, 1988, 1990,
       2004, 1999, 1986, 1992, 2021, 2020, 2023, 2018, 2019, 2022,    0,
       2003, 2025, 1996, 1984, 1994, 1981, 1995, 1991, 1989, 1983, 1987,
       1985, 1980])

In [69]:

values_replace = [7.,0.]
df['Vehicle Model Year']=df['Vehicle Model Year'].replace(values_replace,np.nan)


In [115]:
df['Vehicle Model Year']=df['Vehicle Model Year'].fillna(0)

In [117]:
df['Vehicle Model Year']=df['Vehicle Model Year'].astype(int)

In [132]:
df.tail()

,Public Vehicle Number,Status,Vehicle Make,Vehicle Model,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source,Wheelchair Accessible,Company Name,Address,City,State,ZIP Code,Taxi Affiliation,Taxi Medallion License Management,Record ID
66403,42,ACTIVE,PRECISION,6 SEATER,2020,FUSCIA,Pedal,N,JBL PEDICAB MANAGEMENT LLC,1627 N. TRIPP AVE.,CHICAGO,IL,60639,NaN,NaN,42Pedicab
66404,52,ACTIVE,SCHWINN,NONE,2014,GREEN,Pedal,N,BM PEDICAB INC.,4450 N. KEDZIE AVE.,CHICAGO,IL,60625,NaN,NaN,52Pedicab
66405,70,VIOLATION,MAINSTREET,CLASSIC,0,RED/BLACK,Pedal,N,EMPIRE PEDICAB LLC,318 W. ADAMS ST.,CHICAGO,IL,60606,NaN,NaN,70Pedicab
66406,57,VIOLATION,MAINSTREET,BOARDWALK,2011,RED,Pedal,N,CARGILL C KELLY II,2428 W. 24TH PL.,CHICAGO,IL,60608,NaN,NaN,57Pedicab
66407,117,ACTIVE,SCHWINN,NONE,2014,GREEN,Pedal,N,BM PEDICAB INC.,4450 N. KEDZIE AVE.,CHICAGO,IL,60625,NaN,NaN,117Pedicab


In [133]:
df['Vehicle Make'].unique()

array(['CHEVROLET', 'MERCEDES', 'VAN HOOL', 'VPG', 'DODGE', 'TOYOTA',
       'FORD', 'CADILLAC', 'FREIGHLINE', 'CHAMPION', 'THOMAS',
       'FREIGHTLINER', 'MCI', nan, 'ALEXANDER DENNIS LTD', 'GILLIG',
       'INTL', 'LEYLAND', 'ALEXDER DENNIS LTD', 'IC', 'DOUBLE SHUFLE',
       'ISUZU', 'SPEC-CONSTED', 'TESLA', 'KIA', 'TUK TUK', 'MAINSTREET',
       'LUCID', 'POLARIS', 'BMW', 'PRECISION', 'CHRYSLER', 'MERCURY',
       'LINCOLN', 'GMC', 'ALEXANDER', 'VOLVO', 'HONDA', 'HYUNDAI',
       'NISSAN', 'AUDI', 'LEXUS', 'INFINITI', 'JEEP', 'MOVITRON', 'ORION',
       'ANKAI', 'MASERATI', 'MAZDA', 'CHARLESTON', 'NEOPLAN', 'GENISIS',
       'OSHKOSH', 'BLUEBIRD', 'VOLKSWAGEN', 'BOYERTOWN', 'MITSUBISHI',
       'MOBILITY VENTURE', 'LAND ROVER', 'TURTLE TOP', 'SUBARU', 'ACURA',
       'SCION', 'Dodge', 'Ford', 'PONTIAC', 'BMX', 'BARTH', 'JAGUAR',
       'CINDERELLA', 'TROYER', 'KIMBALL', 'YODER', 'MARTIN', 'SCHWARTZ',
       'MILLER', 'ANDERSON', 'SCHROCK', 'VICTORIA', 'STUDEBAKER',
       'MANMADE

In [123]:
df['ZIP Code']=df['ZIP Code'].fillna(0)

In [125]:
df['ZIP Code']=df['ZIP Code'].astype(int)

In [139]:
df.drop_duplicates(subset=['Public Vehicle Number'],inplace=True)

In [140]:
df

,Public Vehicle Number,Status,Vehicle Make,Vehicle Model,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source,Wheelchair Accessible,Company Name,Address,City,State,ZIP Code,Taxi Affiliation,Taxi Medallion License Management,Record ID
0,12009,RESERVED,CHEVROLET,EXPRESS,2014,BLACK,Bio-Diesel,N,CHICAGO PRIVATE TOURS LLC,4567 S. OAKENWALD AVE.,CHICAGO,IL,60653,NaN,NaN,12009Charter Sightseeing
1,12248,INACTIVE,MERCEDES,SPRINTER,2010,SILVER,Bio-Diesel,N,O'HARE-MIDWAY LIMOUSINE SERVICE INC # 2,4610 N. CLARK ST.,CHICAGO,IL,60640,NaN,NaN,12248Charter Sightseeing
2,13527,INACTIVE,VAN HOOL,TD925,2008,RED,Bio-Diesel,N,"TRT TRANSPORTATION, INC.",4400 S. RACINE AVE.,CHICAGO,IL,60609,NaN,NaN,13527Charter Sightseeing
4,13528,INACTIVE,VAN HOOL,TD925,2008,RED,Bio-Diesel,N,"TRT TRANSPORTATION, INC.",4400 S. RACINE AVE.,CHICAGO,IL,60609,NaN,NaN,13528Charter Sightseeing
5,12025,INACTIVE,MERCEDES,SPRINTER,2015,BLACK,Bio-Diesel,N,O'HARE-MIDWAY LIMOUSINE SERVICE INC # 2,4610 N. CLARK ST.,CHICAGO,IL,60640,NaN,NaN,12025Charter Sightseeing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16409,998,VIOLATION,TOYOTA,CAMRY,2012,WHITE,Hybrid,N,AGOMONY CAB INC,2601 W. PETERSON AVE.,CHICAGO,IL,60659,DIS-AFFILIATED,Owner Manager,998Taxi
16410,3810,ACTIVE,TOYOTA,RAV4,2017,GRAY/WHITE,Hybrid,N,FENKIL ASMERA TAXI COMPANY,820 W. BELLE PLAINE AVE.,CHICAGO,IL,60613,Choice Taxi Association Inc,Owner Manager,3810Taxi
16411,4690,VIOLATION,TOYOTA,CAMRY,2014,WHITE,Hybrid,N,MURIELLE CADICHON,1736 W. 105TH PL.,CHICAGO,IL,60643,DIS-AFFILIATED,Owner-Operator,4690Taxi
16412,2831,ACTIVE,TOYOTA,PRIUS,2012,WHITE,Hybrid,N,G S TRANS INC,2601 W. PETERSON AVE.,CHICAGO,IL,60659,City Service Taxi Association,Owner Manager,2831Taxi


In [141]:
df.drop(columns=['Address'],inplace=True)

In [142]:
df

,Public Vehicle Number,Status,Vehicle Make,Vehicle Model,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source,Wheelchair Accessible,Company Name,City,State,ZIP Code,Taxi Affiliation,Taxi Medallion License Management,Record ID
0,12009,RESERVED,CHEVROLET,EXPRESS,2014,BLACK,Bio-Diesel,N,CHICAGO PRIVATE TOURS LLC,CHICAGO,IL,60653,NaN,NaN,12009Charter Sightseeing
1,12248,INACTIVE,MERCEDES,SPRINTER,2010,SILVER,Bio-Diesel,N,O'HARE-MIDWAY LIMOUSINE SERVICE INC # 2,CHICAGO,IL,60640,NaN,NaN,12248Charter Sightseeing
2,13527,INACTIVE,VAN HOOL,TD925,2008,RED,Bio-Diesel,N,"TRT TRANSPORTATION, INC.",CHICAGO,IL,60609,NaN,NaN,13527Charter Sightseeing
4,13528,INACTIVE,VAN HOOL,TD925,2008,RED,Bio-Diesel,N,"TRT TRANSPORTATION, INC.",CHICAGO,IL,60609,NaN,NaN,13528Charter Sightseeing
5,12025,INACTIVE,MERCEDES,SPRINTER,2015,BLACK,Bio-Diesel,N,O'HARE-MIDWAY LIMOUSINE SERVICE INC # 2,CHICAGO,IL,60640,NaN,NaN,12025Charter Sightseeing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16409,998,VIOLATION,TOYOTA,CAMRY,2012,WHITE,Hybrid,N,AGOMONY CAB INC,CHICAGO,IL,60659,DIS-AFFILIATED,Owner Manager,998Taxi
16410,3810,ACTIVE,TOYOTA,RAV4,2017,GRAY/WHITE,Hybrid,N,FENKIL ASMERA TAXI COMPANY,CHICAGO,IL,60613,Choice Taxi Association Inc,Owner Manager,3810Taxi
16411,4690,VIOLATION,TOYOTA,CAMRY,2014,WHITE,Hybrid,N,MURIELLE CADICHON,CHICAGO,IL,60643,DIS-AFFILIATED,Owner-Operator,4690Taxi
16412,2831,ACTIVE,TOYOTA,PRIUS,2012,WHITE,Hybrid,N,G S TRANS INC,CHICAGO,IL,60659,City Service Taxi Association,Owner Manager,2831Taxi
